In [7]:
import time
import nest_asyncio
import requests
import json
from urllib import request
import multiprocessing
import asyncio
import aiohttp
from concurrent.futures import ThreadPoolExecutor
import multiprocessing_methods as mm

#### Pobieranie obrazów i zapisywanie
[Obrazy](pictures)
<br>
Zapisywanie po 10 obrazów
##### Wersja Synchroniczna

In [8]:
def download_images():
	response = requests.get("https://picsum.photos/v2/list")
	if response.status_code != 200:
		raise AttributeError('GET /tasks/ {}'.format(response.status_code))
	data = json.loads(response.text)[:10]

	pictures=[]
	for s in data:
		pictures.append(s['download_url']+".jpg")
	return pictures

def saveImages(link):
	filename = link.split('/')[4]
	fileformat = 'jpg'
	request.urlretrieve(link, f"pictures/{filename}.{fileformat}")

def main():
    images = download_images()
    for image in images:
        saveImages(image)

start_time = time.time()
main()
duration_sync = time.time() - start_time
print(f"Synchronicznie: {duration_sync}")

Synchronicznie: 13.99209475517273


##### Wersja multithreading

In [9]:
def process_images_threading():
    images = download_images()
    with ThreadPoolExecutor(max_workers=5) as executor:
        executor.map(saveImages,images)

start_time = time.time()
process_images_threading()
duration_thread = time.time() - start_time
print(f"Threading: {duration_thread}")

Threading: 10.575223207473755


##### Asyncio

In [10]:
async def download_images_asyncio(link, session):
	filename = link.split('/')[4]
	fileformat = 'jpg'
	async with session.get(link) as response:
			with open("pictures/{}.{}".format(filename, fileformat), 'wb') as fd:
					async for data in response.content.iter_chunked(1024):
							fd.write(data)

async def main_asyncio():
	images = download_images()

	async with aiohttp.ClientSession() as session:
			tasks=[download_images_asyncio(image,session)for image in images]
			return await asyncio.gather(*tasks)

start_time = time.time()
nest_asyncio.apply()
asyncio.run(main_asyncio())
duration_asyncio = time.time() - start_time
print(f"asyncio: {duration_asyncio}")


asyncio: 11.347293615341187


##### Multiprocessing

In [11]:
def process_images_multiprocessing():
    images = download_images()
    pool = multiprocessing.Pool(multiprocessing.cpu_count())
    pool.map(mm.save_images_multiprocessing,images)

start_time = time.time()
process_images_multiprocessing()
duration_multiprocessing = time.time() - start_time
print(f"Multiprocessing: {duration_multiprocessing}")

Multiprocessing: 11.13468861579895


##### Porównanie

In [12]:
print(f"Synchronicznie: {duration_sync}")
print(f"Threading: {duration_thread}")
print(f"asyncio: {duration_asyncio}")
print(f"Multiprocessing: {duration_multiprocessing}")

Synchronicznie: 13.99209475517273
Threading: 10.575223207473755
asyncio: 11.347293615341187
Multiprocessing: 11.13468861579895
